### Links:

- home: https://feverup.com/madrid
- ejemplo con accesibilidad: https://feverup.com/m/100122

In [37]:
# Libraries

import numpy as np
import pandas as pd
import re
import emoji

import requests
from bs4 import BeautifulSoup
import html
import lxml

## 1) Check whether an event is accesible or not for disabled people

### 1.1) Scrapping url with accesible event

In [46]:
# 1) Pull info from the url
def pull_data(url):
    r = requests.get(url)

    # URL parsing
    soup = BeautifulSoup(r.text, "lxml")

    # Title & Description (info)
    event_title = soup.find(class_ = "plan-hero__title").text
    event_descr = soup.find(class_ = "plan-description mb-32")

    #text = event_descr.text
    # Parsing soup object to string
    text_with_html = str(event_descr)

    return text_with_html, event_title

# 2) Split the info into the different web sections
def section_split(string, splitter):
    # Split items using the given splitter
    info_items = string.split(splitter)
    # To store the new items
    info_items_2 = []
    for item in info_items:
        # To remove html code from each item
        info_items_2.append(re.sub(r"\<.*?\>", "", item))
    return info_items_2

# 3) Pull the information from the general information section
def gen_info_split(iterator:str, icons_list:list):
    # To save the info we extract from the string
    general_info = []

    # Iterate over all the icons in the list
    for icon in icons:
        # It the icon is in the string, then...
        if icon in iterator:
            # Split iterator using the icon and save it as the new iterator
            # Input: string
            # Output: list with 2 items
            iterator = iterator.split(icon)
            # Save the first item in our new list
            general_info.append(iterator[0])

            # Check if there is still any icon left in the second element of the iterator. If so, save it as the new iterator (a string again)
            if any(icon in iterator[1] for icon in icons):
                iterator = iterator[1]
            # Else, save it in our new list.
            # As we don't have more icons in the second element, that means, we don't need to split the iterator anymore, since we already reach the last piece of info we needed
            else:
                general_info.append(iterator[1])
        else:
            pass

    return general_info

In [47]:
url = "https://feverup.com/m/100122"
#url = "https://feverup.com/m/100044"

info, event = pull_data(url)
info_items = section_split(info, "strong")

icons = ["📅", "🕒", "⏳", "👤", "📍", "⚠️", "♿", "⌚", "❓", "🔗"]
info_items[6] = gen_info_split(info_items[6], icons)

In [42]:
full_icons_list = emoji.UNICODE_EMOJI["es"].keys()

In [43]:
any(icon in test for icon in full_icons_list)

True

In [58]:
r = requests.get(url)
soup = BeautifulSoup(r.text, "lxml")
descr = soup.find(class_ = "plan-description mb-32")
descr2 = str(descr)
re.findall(r"\<strong\>.*?\<\/strong\>", descr2)

['<strong>Tickets</strong>',
 '<strong>Qué vas a disfrutar</strong>',
 '<strong>Información General</strong>',
 '<strong>Descripción</strong>']

In [59]:
re.findall(r"\<\/strong\>.*?\<strong\>", descr2)

['</strong><br/>🎫 Entrada - incluye acceso a la exhibición y audioguía<ul><li>General adulto (mayores de 18 años)</li><li>Reducida (diversidad funcional, estudiantes, mayores de 65 años y menores de 18, mostrando siempre documentación)</li><li>Pack de 4 entradas (precio reducido)</li><li>Grupos a partir de 10 personas (precio reducido)</li><li>Menores de 2 años (entrada gratuita)</li></ul><br/><strong>',
 '</strong><br/>🎨 Descubre una nueva forma de admirar las obras del genio Vincent van Gogh<br/>👨\u200d🎨 Sumérgete en las pinturas más conocidas del artista gracias a una experiencia tridimensional<br/>📽️ Disfruta de una experiencia basada en una de las colecciones más importantes de Van Gogh y en una investigación exhaustiva sobre su vida<br/>🖼️ Una experiencia diseñada por el Van Gogh Museum de Ámsterdam<br/>🏴\u200d☠️ Adéntrate en una estimulante actividad con la "Búsqueda del tesoro para niños"<br/><br/><strong>',
 '</strong><br/>📅 Fechas: a partir de septiembre de 2021<br/>🕒 Horario

In [65]:
[desc.split("</strong>") for desc in descr2.split("<strong>")]

[['<div class="plan-description mb-32" data-cy="plan-content-description">⭐ <b>Meet Vincent van Gogh</b> es una experiencia tridimensional dividida en seis escenarios que permiten adentrarse en la vida de uno de los pintores más famosos de la historia<br/><br/>'],
 ['Tickets',
  '<br/>🎫 Entrada - incluye acceso a la exhibición y audioguía<ul><li>General adulto (mayores de 18 años)</li><li>Reducida (diversidad funcional, estudiantes, mayores de 65 años y menores de 18, mostrando siempre documentación)</li><li>Pack de 4 entradas (precio reducido)</li><li>Grupos a partir de 10 personas (precio reducido)</li><li>Menores de 2 años (entrada gratuita)</li></ul><br/>'],
 ['Qué vas a disfrutar',
  '<br/>🎨 Descubre una nueva forma de admirar las obras del genio Vincent van Gogh<br/>👨\u200d🎨 Sumérgete en las pinturas más conocidas del artista gracias a una experiencia tridimensional<br/>📽️ Disfruta de una experiencia basada en una de las colecciones más importantes de Van Gogh y en una investigac